This is an animated version of E4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML, display


def create_potential_animation(q=1, s=1, n=3, num_frames=100):
    # Set up the figure
    plt.rcParams['figure.figsize'] = [15, 6]
    fig = plt.figure()
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)

    # Calculate potentials (one-time calculation)
    theta0 = np.pi / n
    R = s / (2 * np.sin(theta0))

    def potential(q, x, y, x0, y0):
        k = 9e9
        r = np.sqrt((x - x0) ** 2 + (y - y0) ** 2 + 1e-9)
        return k * q / r

    X = np.linspace(-5, 5, 200)
    Y = np.linspace(-5, 5, 200)
    x, y = np.meshgrid(X, Y)

    Xm = np.array([R * np.cos(2 * m * theta0) for m in range(n)])
    Ym = np.array([R * np.sin(2 * m * theta0) for m in range(n)])

    Vnet = sum(potential(q, x, y, xm, ym) for xm, ym in zip(Xm, Ym))
    epsilon = 1e-7
    Vplot = np.sign(Vnet) * np.log10(np.abs(Vnet) + epsilon)

    # Create the surface plot (will remain static)
    surf = ax1.plot_surface(x, y, Vplot, cmap='magma', edgecolor='none',
                            antialiased=True, rcount=100, ccount=100, alpha=0.9)

    # Set up 3D plot properties
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel(r'$\log(V)$')
    ax1.set_title('3D Potential with Moving Slice')
    ax1.view_init(elev=25, azim=45)
    ax1.set_box_aspect([1, 1, 0.8])

    # Create slice plane data
    slice_x = np.linspace(-6, 6, 50)
    slice_z = np.linspace(Vplot.min(), Vplot.max(), 50)
    slice_X, slice_Z = np.meshgrid(slice_x, slice_z)

    # Initialize plots that will be updated
    slice_plane = None
    intersection_line = None
    potential_line = None

    # Animation update function
    def update(frame):
        nonlocal slice_plane, intersection_line, potential_line

        # Calculate y position for this frame
        y_slice = np.linspace(-5, 5, num_frames)[frame]

        # Remove previous plots
        if slice_plane is not None:
            slice_plane.remove()
        if intersection_line is not None:
            intersection_line.remove()
        if potential_line is not None:
            potential_line.remove()

        # Update slice plane
        slice_Y = np.full_like(slice_X, y_slice)
        slice_plane = ax1.plot_surface(slice_X, slice_Y, slice_Z,
                                       alpha=0.2, color='yellow', shade=False)

        # Update intersection line
        y_idx = np.abs(Y - y_slice).argmin()
        intersection_line = ax1.plot(X, [y_slice] * len(X), Vplot[y_idx, :],
                                     color='red', linewidth=2)[0]

        # Update 2D plot
        ax2.clear()
        potential_line = ax2.plot(X, Vplot[y_idx, :], 'b-', linewidth=2)[0]
        ax2.grid(True)
        ax2.set_xlabel('x')
        ax2.set_ylabel(r'$\log(V)$')
        ax2.set_title(f'Potential at y = {y_slice:.2f}')
        ax2.set_xlim(-6, 6)
        ax2.set_ylim(np.min(Vplot), np.max(Vplot))
        ax2.axhline(y=0, color='r', linestyle='--', alpha=0.5)

        # Add vertical lines for charges near slice
        for xm, ym in zip(Xm, Ym):
            if abs(ym - y_slice) < 0.2:
                ax2.axvline(x=xm, color='g', linestyle=':', alpha=0.5)

    plt.tight_layout()

    plt.rcParams['animation.embed_limit'] = 50.0

    # Create animation
    anim = FuncAnimation(fig, update, frames=num_frames,
                         interval=25,  # 50ms between frames
                         blit=False,
                         repeat=True)

    return HTML(anim.to_jshtml())


# Create and display the animation
display(create_potential_animation(q=1, s=1, n=12, num_frames=200))
